<a href="https://colab.research.google.com/github/margaretxie/Test_Paper3_with_UCO/blob/main/NOTRUNYET-round2_P3_UCO_v1_selected_KP_selected_Action_new_groundtruth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#test paper 3 - the HK one - with UCO dataset (only cam234, not cam 0 1, total 1296 videos )

# Paper 3 - https://link.springer.com/article/10.1007/s13755-023-00258-3  -skeleton based repeptition count method
# The paper's code - webpage: https://github.com/YuChengHSU/repetition-counting/tree/main

# UCO dataset [paper20] - downloaded from  http://rabinf24.uco.es/avaphy/    (Outlook Emailed got access )

## round2 - v1 - univeral fitting - selected KP with selected Action with new groundtruth, based on v12 and round2_v0
## Round2 - have new groundtruth data

# v18: same as v17, only not 3 picked KPs but 1 KP - 29
# v17: same as v16, only not 33 KP but 3 picked KPs [25, 29, 31]
# v16: only UCO-Action1 (27 person, all ground truth are 4, except person 13's three records are 5), wins= 256, step = 125, KP 33, for comparing with V17, 3 picked KPs

# v14: v13_based, testing action 16 with selection2KPs, with obo & mae, with changing steps to 125
# v13: v9_v10_v11 based, for testing action 16 with hand picked KPs, with final obo & mae

# v12: v9_based, for testing action 16

# v9: v6_based,for testing KPselection- only 81 videos of action 16 "a16", and only with KPs from 13-22 (10 KPs)

# v8: v4_based, 33KPs_obo, mea by type
# v7: v3_based, 33KPs_ merge_estimated_n_groundtrue_count
# v6: v2_based, apply original paper3 model (using whole 33 KPs)
# v5: average KP used, popular KPs analyst
# v4: calculate obo, mea according to action types (1-16)
# v3: merge_estimated_n_groundtrue_count
# v2: apply paper3's model to filtered KPs which visibility >=threshold
# v1: extract each videos' KPs  (this code only for half cameras (2,3,4))
# v0: framework of looping all the .mp4 files, with name PnAn (Person*Action*)#

# Setup
v="R2_V1"

# Import necessary libraries
from google.colab import drive
import pandas as pd
import numpy as np
import os
import subprocess
from datetime import datetime
import pytz

# Mount Google Drive
drive.mount('/content/drive')

results = []

base_path = '/content/drive/My Drive/repetition_dataset/UCO/extracedKP_npy_cam234'
KP33_xyz_path = '/content/drive/My Drive/repetition_dataset/UCO/npy_cam234_33KPs_xyz'
save_path = '/content/drive/My Drive/repetition_dataset/UCO/results_cam234_a16_only'
groundtruth = '/content/gdrive/My Drive/repetition_dataset/UCO/merged_UCO/0_Manually check groud truth number_D.xlsx'  # real ground truth table
df = pd.read_excel(groundtruth)

f=25 # frame rate

wins = "256"
step = "1"

# KP - hand picked
kp_indices = [12, 14, 16, 24, 26, 28]

# FUNCTION for getting the UCO dataset real ground truth
import re

#base_path = '/content/gdrive/My Drive/repetition_dataset/UCO/merged_UCO/clips_mp4'


def get_count_from_excel(video_name, df):
    # Parse person number and action number from video name
    match = re.match(r'p(\d+)a(\d+)c', video_name)
    if not match:
        raise ValueError(f"Invalid video name format: {video_name}")

    person_number = int(match.group(1))  # Extracted person number
    action_number = match.group(2)  # Extracted action number as string to match column name format

    # Construct the column name in the format 'A2', 'A3', etc.
    action_column = f'A{action_number}'

    # Fetch the count from the DataFrame
    try:
        count = df.loc[df['Person'] == person_number, action_column].values[0]
    except KeyError:
        raise ValueError(f"Action code '{action_column}' not found in the Excel file.")
    except IndexError:
        raise ValueError(f"Person number '{person_number}' not found in the Excel file.")

    return count

# testing
video_name = "p1a12c3"
count = get_count_from_excel(video_name, df)
print(f'The count for {video_name} is: {count}')


def get_modified_filename(filename):
    # Find the position of the last underscore
    last_underscore_index = filename.rfind('_')
    # Extract the part of the filename after the last underscore
    if last_underscore_index != -1:
        new_filename = filename[last_underscore_index + 1:]
    else:
        new_filename = filename
    return new_filename


# Ensure the output directory exists
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Function to process videos
def process_videos(base_path, kp_indices):
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if 'a14' in file:
                video_name = get_modified_filename(file)
                video_name = video_name.replace('.mp4', '.npy')
              #  print(f"Processing video: {video_name}")
                file_path = os.path.join(root, file)
                print(f"Processing {file_path}")
                data = np.load(file_path)

                # Use only selected KP
                filtered_data = data[:, kp_indices, :]

                # Save filtered data to a temporary .npy file
                temp_output_path = os.path.join(save_path, 'temp_filtered_data.npy')
                np.save(temp_output_path, filtered_data)

                # Prepare the command to run the external Python script with the necessary arguments
                command = [
                    "python", "rep-count.py",
                    "--data", temp_output_path,
                    "-j", str(len(kp_indices)),
                    "-d", "3",  # Only three dimensions are considered (x, y, z)
                    "-f", str(f),
                    "--wins", wins #"256",
                    "--noverlap", step #"1"
                ]

                # Execute the command and capture the output
                try:
                    result = subprocess.run(command, check=True, capture_output=True, text=True)
                    output = result.stdout.strip()
                    estimated_count = int(output.split()[-1])  # Parse the count from the output
                    print("Output:", output)
                except subprocess.CalledProcessError as e:
                    print("Failed to execute command:", e)
                    print("Error output:", e.stderr)
                    estimated_count = None

                # get groundtruth
                count = get_count_from_excel(video_name, df)


                # Append results to the list
                results.append([video_name, len(kp_indices), estimated_count, data.shape[0]])

# # KP - hand picked
# kp_indices = [12, 14, 16, 24, 26, 28]

process_videos(KP33_xyz_path, kp_indices)

# Save the estimated counts of the filtered videos
current_time_str = datetime.now(pytz.timezone('America/New_York')).strftime('%Y%m%d%I%M%p').lower()
file_name = f"{v}_Cam234_output_{current_time_str}.csv"
output_csv_path = os.path.join(save_path, file_name)

results_df = pd.DataFrame(results, columns=['Video Name', 'KP Numbers', 'Estimated Count', 'Frame Number'])
results_df.to_csv(output_csv_path, index=False)

print("\n########First five rows of results_df:\n")
print(results_df.head(5))

print("\n########Last five rows of results_df:\n")
print(results_df.tail(5))

print(f"\nResults saved to {output_csv_path}")


Mounted at /content/drive
Processing /content/drive/My Drive/repetition_dataset/UCO/npy_cam234_33KPs_xyz/33KP_xyz_p12a14c2.npy
Output: Namespace(data='/content/drive/My Drive/repetition_dataset/UCO/results_cam234_a16_only/temp_filtered_data.npy', j=6, d=3, f=25, wins=256, noverlap=1)
Estimated count 5
Processing /content/drive/My Drive/repetition_dataset/UCO/npy_cam234_33KPs_xyz/33KP_xyz_p12a14c3.npy
Output: Namespace(data='/content/drive/My Drive/repetition_dataset/UCO/results_cam234_a16_only/temp_filtered_data.npy', j=6, d=3, f=25, wins=256, noverlap=1)
Estimated count 5
Processing /content/drive/My Drive/repetition_dataset/UCO/npy_cam234_33KPs_xyz/33KP_xyz_p12a14c4.npy
Output: Namespace(data='/content/drive/My Drive/repetition_dataset/UCO/results_cam234_a16_only/temp_filtered_data.npy', j=6, d=3, f=25, wins=256, noverlap=1)
Estimated count 5
Processing /content/drive/My Drive/repetition_dataset/UCO/npy_cam234_33KPs_xyz/33KP_xyz_p13a14c2.npy
Output: Namespace(data='/content/drive/My 